In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [3]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
def clean_reviews(no_reviews,dataset):
    import re
    import nltk
    nltk.download('stopwords') 
    from nltk.corpus import stopwords 
    
    my_stopwords = stopwords.words('english') 
            
    from nltk.stem.porter import PorterStemmer

     
    corpus =[] 
    for i in range (0, no_reviews):
        review = re.sub('[^a-zA-Z]',' ', dataset['review'][i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() 
        review = review.split()
        ps = PorterStemmer() 
        
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)]
        review = ' '.join(review) 
        corpus.append(review)
    return corpus

In [5]:
x = clean_reviews(50000,df)

[nltk_data] Downloading package stopwords to /home/aayush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def bag_of_words(corpus, maximum_features):
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(ngram_range=(1,3), binary=True)
    X = vect.fit_transform(corpus)
    return X

In [7]:
x

['complet appal see averag rate movi affect definit one worst movi ever seen still keep wonder watch end first plot total hopeless act truli aw think total unknown actress would better role susan lucci concern mr kamar del rey think would better choic remain valley doll sum total wast time tri stay polit avoid cost rate still think well paid sinc cannot give',
 'movi requir view librarian would librarian best line directli relat librarianship public librari vs academ librari argument classic argument wage among librarian librari school student also break mani librarian stereotyp librarian might even capabl fun even usual sex romanc languag section best movi librarian desk set katherin hepburn spencer traci cours',
 'spoiler alert br br never seen comment movi disagre comment peopl made one could learn critic view movi educ film rate highli teach succeed watch movi learn gener watch entertain entertain rate low end downbeat cerebr intellectu conflict result eight star rate movi follow j

In [8]:
x_data = bag_of_words(x,1000)

In [9]:
x_data

<50000x7869861 sparse matrix of type '<class 'numpy.int64'>'
	with 16493611 stored elements in Compressed Sparse Row format>

In [10]:
labels = df['label']
labels = labels.map({'pos': 1, 'neg': 0})

In [11]:
train_data = x_data[0:34999,:]
test_data = x_data[35000:49999,:]
train_labels = labels[0:34999]
test_labels = labels[35000:49999]

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100)

In [14]:
clf.fit(train_data, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
y_pred_class = clf.predict(test_data)

In [16]:
from sklearn import metrics
metrics.accuracy_score(test_labels, y_pred_class)

0.8561237415827722

In [17]:
metrics.confusion_matrix(test_labels, y_pred_class)

array([[6136, 1311],
       [ 847, 6705]])